Importing The Libraries Required

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
import torch
import os
from torch.utils.data import DataLoader


#**Data pre processing step and processing step**


In [13]:
# 1. Define Constants
TRAIN_DIR = 'archive (3) - Copy/train'  
TEST_DIR = 'archive (3) - Copy/test'  
IMG_SIZE = 48               
BATCH_SIZE = 64

# 2. Define Transforms (The "Preprocessing" Step)
# This replaces your manual normalization and reshaping
data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1), # Forces 1 channel (Grayscale)
    transforms.Resize((IMG_SIZE, IMG_SIZE)),     # Resizes to 48x48
    transforms.ToTensor(),                       # Converts to Tensor & Scales to [0, 1]
])

# 3. Load the Images from Folders
train_data = datasets.ImageFolder(root=TRAIN_DIR, transform=data_transforms)
test_data = datasets.ImageFolder(root=TEST_DIR, transform=data_transforms)

# 4. Create DataLoaders (The "Feeder")
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

# --- SANITY CHECK (Run this once to verify) ---
print(f"Classes found: {train_data.classes}")
images, labels = next(iter(train_loader))
print(f"Batch shape: {images.shape}") 
print(train_loader)
# Should print: torch.Size([64, 1, 48, 48]) -> (Batch, Channel, Height, Width)

Classes found: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Batch shape: torch.Size([64, 1, 48, 48])


**Building The CNN Model**

In [14]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        # Conv Block 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        # Conv Block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        # Conv Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(128 * 6 * 6, 128) 
        self.fc2 = nn.Linear(128, 7) # 7 Classes
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 6 * 6) # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

**Applying CNN**

In [ ]:
import torch.optim as optim

# 1. Setup Device 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

# 2. Instantiate Model, Loss, and Optimizer
model = EmotionCNN().to(device) # Move model to GPU/CPU
criterion = nn.CrossEntropyLoss() # Handles Softmax automatically
optimizer = optim.Adam(model.parameters(), lr=0.001) # standard learning rate

# 3. The Training Function
def train_model(num_epochs):
    print("Starting Training...")
    
    for epoch in range(num_epochs):
        model.train() # Set to training mode
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            
            images, labels = images.to(device), labels.to(device)
            
            # A. Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # B. Backward Pass & Optimization
            optimizer.zero_grad() # Clear old gradients
            loss.backward()       # Calculate new gradients
            optimizer.step()      # Update weights
            
            # C. Track Stats
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        # Print stats after every epoch
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] -> Loss: {running_loss/len(train_loader):.4f} | Accuracy: {epoch_acc:.2f}%")

train_model(num_epochs=50)

Training on: cpu
Starting Training...
Epoch [1/50] -> Loss: 1.7219 | Accuracy: 30.18%
Epoch [2/50] -> Loss: 1.5318 | Accuracy: 40.79%
Epoch [3/50] -> Loss: 1.4349 | Accuracy: 44.92%
Epoch [4/50] -> Loss: 1.3531 | Accuracy: 48.33%
Epoch [5/50] -> Loss: 1.2865 | Accuracy: 50.96%
Epoch [6/50] -> Loss: 1.2387 | Accuracy: 53.10%
Epoch [7/50] -> Loss: 1.1931 | Accuracy: 54.51%
Epoch [8/50] -> Loss: 1.1469 | Accuracy: 56.48%
Epoch [9/50] -> Loss: 1.1048 | Accuracy: 58.18%
Epoch [10/50] -> Loss: 1.0653 | Accuracy: 59.39%
Epoch [11/50] -> Loss: 1.0341 | Accuracy: 60.62%
Epoch [12/50] -> Loss: 1.0052 | Accuracy: 61.51%
Epoch [13/50] -> Loss: 0.9649 | Accuracy: 63.04%
Epoch [14/50] -> Loss: 0.9381 | Accuracy: 63.97%
Epoch [15/50] -> Loss: 0.8984 | Accuracy: 65.55%
Epoch [16/50] -> Loss: 0.8745 | Accuracy: 65.82%
Epoch [17/50] -> Loss: 0.8466 | Accuracy: 67.10%
Epoch [18/50] -> Loss: 0.8141 | Accuracy: 68.11%
Epoch [19/50] -> Loss: 0.7901 | Accuracy: 69.19%
Epoch [20/50] -> Loss: 0.7626 | Accuracy

In [24]:
def check_accuracy(loader, model):
    print("Checking accuracy...") # Simple print to start
    
    num_correct = 0
    num_samples = 0
    model.eval()  # Set model to evaluation mode
    
    with torch.no_grad():
        for x, y in loader:
           
            x = x.to(device)
            y = y.to(device)
            
            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)
            
            # Count correct
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
    # Calculate math
    acc = float(num_correct) / float(num_samples) * 100
    print(f'Got {num_correct} / {num_samples} correct ({acc:.2f}%)')
    
    model.train() 
    return acc


print("--- TEST DATA RESULTS ---")
check_accuracy(test_loader, model)
#

--- TEST DATA RESULTS ---
Checking accuracy...
Got 4005 / 7178 correct (55.80%)


55.79548620785734

In [25]:
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pandas as pd

def evaluate_dataset(model, loader, dataset_name, classes):
    """
    Runs the model on a dataset and prints detailed metrics.
    """
    model.eval()  # Set model to evaluation mode
    y_true = []
    y_pred = []
    
    print(f"\n{'='*20} Evaluating {dataset_name} Set {'='*20}")
    
    # Disable gradient calculation for speed
    with torch.no_grad():
        for images, labels in loader:
            # Move data to the same device as model
            images = images.to(device) 
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            # Store results (move to CPU and convert to numpy)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # --- 1. Overall Accuracy ---
    acc = accuracy_score(y_true, y_pred)
    print(f"Overall Accuracy: {acc*100:.2f}%")
    
    # --- 2. Detailed Metrics (Precision, Recall, F1) ---
    print("\n--- Detailed Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=classes, zero_division=0))
    
    # --- 3. Confusion Matrix (Where is the model getting confused?) ---
    # Convert to DataFrame for better readability
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=[f"True {c}" for c in classes], 
                         columns=[f"Pred {c}" for c in classes])
    
    print("\n--- Confusion Matrix (Row=True, Col=Pred) ---")
    print(cm_df)
    
    return acc

# --- CONFIGURATION ---
# Ensure 'device' is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ensure 'classes' matches your training folder structure exactly
# Example: classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
# If you used ImageFolder, you can get this via: train_dataset.classes
print(f"Classes detected: {classes}")

# --- RUN EVALUATION ---

# 1. Check Training Data (To see if it learned at all)
print("Checking Training Data...")
train_acc = evaluate_dataset(model, train_loader, "TRAINING", classes)

# 2. Check Testing Data (To see if it generalizes)
print("Checking Testing Data...")
test_acc = evaluate_dataset(model, test_loader, "TESTING", classes)

# --- DIAGNOSIS ---
print("\n" + "="*50)
print(f"FINAL DIAGNOSIS:")
if train_acc > 0.90 and test_acc < 0.50:
    print(">> OVERFITTING DETECTED.")
    print("The model memorized the training images but fails on new ones.")
    print("Try: Dropout layers, Data Augmentation, or fewer epochs.")
elif train_acc < 0.50:
    print(">> UNDERFITTING DETECTED.")
    print("The model isn't learning well enough yet.")
    print("Try: More epochs, a more complex model (ResNet), or checking learning rate.")
else:
    print(">> Metrics look balanced (or both are equally average).")
print("="*50)

Classes detected: ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
Checking Training Data...

==================== Evaluating TRAINING Set ====================
Overall Accuracy: 93.99%

--- Detailed Classification Report ---
              precision    recall  f1-score   support

       Angry       0.92      0.91      0.92      3995
     Disgust       0.99      0.92      0.95       436
        Fear       0.93      0.88      0.90      4097
       Happy       0.98      0.98      0.98      7215
     Neutral       0.93      0.95      0.94      4965
         Sad       0.89      0.94      0.91      4830
    Surprise       0.97      0.96      0.97      3171

    accuracy                           0.94     28709
   macro avg       0.94      0.93      0.94     28709
weighted avg       0.94      0.94      0.94     28709


--- Confusion Matrix (Row=True, Col=Pred) ---
               Pred Angry  Pred Disgust  Pred Fear  Pred Happy  Pred Neutral  \
True Angry           3630       

--- REAL-TIME EMOTION CHECK ---
Testing Model on 10 Random Images...

Img 1: True: Neutral    | Pred: Sad        (42.5%) -> WRONG  
Img 2: True: Happy      | Pred: Happy      (100.0%) -> CORRECT
Img 3: True: Sad        | Pred: Neutral    (72.4%) -> WRONG  
Img 4: True: Happy      | Pred: Fear       (86.7%) -> WRONG  
Img 5: True: Angry      | Pred: Angry      (87.1%) -> CORRECT
Img 6: True: Neutral    | Pred: Fear       (69.0%) -> WRONG  
Img 7: True: Surprise   | Pred: Fear       (50.4%) -> WRONG  
Img 8: True: Happy      | Pred: Sad        (76.7%) -> WRONG  
Img 9: True: Sad        | Pred: Neutral    (88.5%) -> WRONG  
Img 10: True: Happy      | Pred: Happy      (100.0%) -> CORRECT
--------------------------------------------------
Mini-Batch Accuracy: 3/10
